In [48]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model

Test pobranego modelu w całości - przy wykorzystywaniu ImageNet mamy 1000 klas

In [49]:
#Import model
model = VGG16()

In [50]:
print(model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [51]:
from keras.preprocessing.image import load_img
# load an image from file
image = load_img('wasp.jpg', target_size=(224, 224))
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

In [52]:
# predict the probability across all output classes
yhat = model.predict(image)

In [53]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

bee (32.78%)


Zadanie 1a

In [54]:
from pathlib import Path


train_dir = "kaggle_bee_vs_wasp/"
bs = 64 # Batch size
resize_size = 96 # for training, resize all the images to a square of this size
training_subsample = 0.1 # for development, use a small fraction of the entire dataset rater than full dataset



bees_vs_wasps_dataset_path=Path(train_dir) # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)
insect_class = {'bee': 0,'wasp': 1, 'insect': 2,'other': 3} 
#df_labels.label = [insect_class[item] for item in df_labels.label] 
df_labels = df_labels[['path','label']]

In [55]:
train_df, validation_df = train_test_split(df_labels, test_size=0.1)
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)

In [56]:
pretrained_model = VGG16(include_top=False, weights= "imagenet",input_shape=(resize_size, resize_size, 3))
print(pretrained_model.summary())
for layer in pretrained_model.layers:
    layer.trainable = False

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0     

In [57]:
last_layer = pretrained_model.get_layer('block5_pool')
last_output = last_layer.output
     
x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

model = Model(pretrained_model.input, x)

In [58]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)      

In [59]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_dir,  
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=bs
)


Found 1027 validated image filenames belonging to 4 classes.


In [60]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=bs
)

Found 115 validated image filenames belonging to 4 classes.


In [61]:
n_training_samples = len(train_df)
n_validation_samples = len(validation_df)

history = model.fit_generator(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=n_validation_samples//bs,
    steps_per_epoch=n_training_samples//bs)

Epoch 1/25
16/16 [==============================] - 13s 824ms/step - loss: 1.2646 - accuracy: 0.4465 - val_loss: 1.3193 - val_accuracy: 0.3594
Epoch 2/25
16/16 [==============================] - 13s 843ms/step - loss: 1.1986 - accuracy: 0.4642 - val_loss: 1.2812 - val_accuracy: 0.3906
Epoch 3/25
16/16 [==============================] - 13s 802ms/step - loss: 1.1634 - accuracy: 0.5202 - val_loss: 1.2728 - val_accuracy: 0.4219
Epoch 4/25
16/16 [==============================] - 13s 819ms/step - loss: 1.1267 - accuracy: 0.5369 - val_loss: 1.2214 - val_accuracy: 0.3906
Epoch 5/25
16/16 [==============================] - 14s 884ms/step - loss: 1.1206 - accuracy: 0.4984 - val_loss: 1.1728 - val_accuracy: 0.4375
Epoch 6/25
16/16 [==============================] - 14s 851ms/step - loss: 1.0954 - accuracy: 0.5576 - val_loss: 1.1311 - val_accuracy: 0.4531
Epoch 7/25
16/16 [==============================] - 13s 840ms/step - loss: 1.0676 - accuracy: 0.5670 - val_loss: 1.1800 - val_accuracy: 0.4844

In [62]:
history.history["val_loss"]

[1.3192517757415771,
 1.2812132835388184,
 1.272782325744629,
 1.2214078903198242,
 1.1727538108825684,
 1.1311153173446655,
 1.1799753904342651,
 1.23663330078125,
 1.2160691022872925,
 1.0819427967071533,
 1.0010037422180176,
 1.0843676328659058,
 1.0721657276153564,
 0.9744089841842651,
 1.0431559085845947,
 1.0349478721618652,
 0.999741792678833,
 1.1782153844833374,
 1.0053977966308594,
 1.134765386581421,
 1.0752544403076172,
 0.9743689894676208,
 1.0440798997879028,
 1.0578973293304443,
 1.080617904663086]